# Sleep Stage Transition Analysis

This Jupyter notebook is designed to analyze the simulation output of the thalamocortical network model described in the paper "Cellular and neurochemical basis of sleep stages in the thalamocortical network" by Krishnan et al. The notebook guides you through the process of setting up the environment, running the simulation, loading the output data, and performing various analyses to understand the transitions between different sleep stages.

## Setup and Installation

You can use the pre-installed code on Cybershuttle or you can install the C++ code locally. The simulation code is implemented in C++ and uses OpenMP for parallelism. You will need a C++ compiler and OpenMP support.

## Cloning the Repository

Clone the repository to your local machine using the following command:
```bash
git clone https://github.com/bazhlab-ucsd/sleep-stage-transition.git
```

## Building the Project

Navigate to the project directory and compile the code:
```bash
cd sleep-stage-transition
make network
make run
```


## Running Simulations

The simulation parameters can be modified in the `params.txt` file. Adjust this file to set different levels of neuromodulators and other parameters. Run the simulation with:
```bash
make run
```
# Alternatively you can run the code using Cybershuttle

## Loading and Processing Output Data

After running the simulation, the output files will be saved in the `out` directory. These files contain the membrane voltage and other relevant data of different neuron types.

# For simulations ran through Cybershuttle, the Cybershuttle local agent can transparently fetch these output files

In [1]:
import numpy as np
import matplotlib.pylab as plt

### Functions

#### Simple Plot Function
This function visualizes the simulation output of the thalamocortical network model. It creates image plots of the membrane potentials of cortical, thalamic, and reticular neurons, as well as time-series plots of specific neurons' membrane potentials.

In [2]:
def gen_cx_tc_re_imageplot(spath):
    cx = np.loadtxt(spath + 'time_cx')
    tc = np.loadtxt(spath + 'time_tc')
    re = np.loadtxt(spath + 'time_re')

    plt.figure(figsize=(10,5))
    plt.subplot(3,2,1)
    plt.imshow(cx[:,1:-1].T, aspect='auto',vmin=-80,vmax=-50)

    plt.subplot(3,2,3)
    plt.imshow(tc[:,1:-1].T, aspect='auto',vmin=-80,vmax=-50)

    plt.subplot(3,2,5)
    plt.imshow(re[:,1:-1].T, aspect='auto',vmin=-80,vmax=-50)

    plt.subplot(3,2,2)
    plt.plot(cx[:,200])

    plt.subplot(3,2,4)
    plt.plot(tc[:,50])

    plt.subplot(3,2,6)
    plt.plot(re[:,50])
    return cx,tc,re

### Compute Mean as Local Field Potential (LFP)
This function provides a way to compute and visualize the LFP from the membrane potentials of cortical neurons, giving insights into the overall electrical activity in the simulated brain region.

In [4]:
def compute_mean_as_lfp(time_cx):
  # Compute mean from index 1 to -1, assuming 0 is cell number and last is predefined
  return np.mean(time_cx[:,1:-1])

#### Compute LFP
The function fftc performs a Fast Fourier Transform (FFT) on the LFP data to analyze the frequency components of the signal. This function provides a method to analyze the frequency components of the LFP signal, which can reveal insights into the underlying neuronal dynamics and oscillatory activity.

In [9]:
! pip install scipy
import scipy

from scipy import signal

def fftc(data, fs, pad, h_freq ):
    # Usage : fft_corrected(data, fs, win, pad )
    #         data  --- np array
    #         fs  --- sampling rate
    #         pad  --- padding
    # need to load numpy as np

    # if win.shape[0]>0:
    #     data=np.multiply(data,win)

    pad = data.shape[0]
    fft_out=np.fft.fft(data,n=pad)

    # frequency=np.multiply(np.arange(0,pad/2),fs/pad) 

    frequency=np.linspace(0.0, 1.0/(2.0/fs), pad/2)
    # frequency=np.fft.fftfreq(data.shape[-1])

    power=np.absolute(fft_out)
    phase=np.angle(fft_out);

    h_freq_bin = np.where(frequency<h_freq)    
    power=power[h_freq_bin]/np.shape(data)[0];
    phase=phase[h_freq_bin];
    frequency=frequency[h_freq_bin];

    return power, phase, frequency

ModuleNotFoundError: No module named 'scipy'